- logits almost at zero at initialization => init to smaller params values
- saturation of tanh => why we don't want tails distributions (dead neurons that never activate and update less frequently) [ occurs for activation function with flat tails ]
- initializing tensor with kaiming normal

https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part3_bn.ipynb

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd

### MLP Revisited